In [1]:
#@title Setup notebook
# Colab setup
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade biocircuits bokeh-catplot watermark blackcellmagic"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

#jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' \ --port=9090 --no-browser

# % pip install biocircuits bokeh-catplot watermark blackcellmagic
import multiprocessing
import tqdm

import numpy as np
import scipy.stats as st
import numba
import math
import pandas as pd

from scipy.signal import argrelextrema
import scipy.fftpack
import os.path

import biocircuits

# Plotting modules
import bokeh.io
import bokeh.plotting
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.io import export_svgs
from bokeh.models import Range1d, DataRange1d

bokeh.io.output_notebook()
from bokeh.io import output_notebook
output_notebook()

# increase max figure height to prevent nested vertical scrooling
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))


<IPython.core.display.Javascript object>

# Stochastic modeling of Aca2 system

The following document describe the modeling and analysis of the anti-CRISPR-associated proteins (Aca2).

We introduce the following notation for the Aca2 system. A gene encodes for AcrlF8 ($Y$) and Aca2 dimer ($X$). This gene transcripte  an mRNA molecule ($M$). This mRNA can translate the proteins $Y$ and $X$ at different rates. The protein $X$ can bind close to the promoter $U$ and form a new complex $U_x$. As a result, it can represss transcription. It also can bind to the mRNA and form a new complex $M_x$. As a consequence, it only inhibits translation of $Y$ but not $X$. We summarize the chemical reactions below:

\begin{align}
U \xrightarrow[]{\alpha} U + M && M  \xrightarrow{\phi} \emptyset \\
M \xrightarrow{\beta} M + X && X  \xrightarrow{\delta} \emptyset \\
M \xrightarrow{\theta} M + Y && Y  \xrightarrow{\delta} \emptyset \\
X + M \xrightarrow[]{a} M_x && M_x  \xrightarrow{d} X + M \\
M_x \xrightarrow[]{\beta} M_x + X &&   \\
X + U \xrightarrow[]{a_u} U_x && U_x  \xrightarrow{d_u} X + U
\end{align}

Next, we model the DNA replication and use population dynamics to describe the exponential grow
\begin{align}
U \xrightarrow[]{f} U + U && U_x  \xrightarrow{} U + X
\end{align}
where $f = r \left (1 - \frac{u+u_x}{u^{tot}} \right )$.

In [2]:
#@title Events and propensities matricies
simple_update = np.array(
  [ # u,  ux, m,  mx, x,  y
    [ 1,  0,  0,  0,  0,  0], # Unbound DNA replication
    [ 2, -1,  0,  0,  1,  0], # Bound DNA replication and displacement of Aca2
    [-1,  1,  0,  0, -1,  0], # Aca2 binding DNA
    [ 1, -1,  0,  0,  1,  0], # Aca2 dissociating from DNA
    [ 0,  0,  1,  0,  0,  0], # Transcription of unbound DNA
    [ 0,  0, -1,  1, -1,  0], # Aca2 binding RNA
    [ 0,  0,  1, -1,  1,  0], # Aca2 dissociating from RNA
    [ 0,  0, -1,  0,  0,  0], # free RNA decay
    [ 0,  0,  0, -1,  1,  0], # Bound RNA decay and release of Aca2
    [ 0,  0,  0,  0,  1,  0], # Aca2 translation from free RNA
    [ 0,  0,  0,  0,  1,  0], # Aca2 translation from bound RNA
    [ 0,  0,  0,  0, -1,  0], # Aca2 decay
    [ 0,  0,  0,  0,  0,  1], # Acr translation from free RNA
    [ 0,  0,  0,  0,  0, -1], # Acr decay
    ],
    dtype=int,
 )

def simple_propensity(propensities, state, t, alpha, beta, theta, phi, delta, am, dm, au, du, umax, r, latency):
    u, ux, m, mx, x, y = state
    propensities[0]  = ((t - latency) > 0) * r*(1 - (u + ux)/umax)*u   # Unbound DNA replication
    propensities[1]  = ((t - latency) > 0) * r*(1 - (u + ux)/umax)*ux  # Bound DNA replication and displacement of Aca2
    propensities[2]  = au*u*x                    # Aca2 binding DNA
    propensities[3]  = du*ux                     # Aca2 dissociating from DNA
    propensities[4]  = alpha*u                   # Transcription of unbound DNA
    propensities[5]  = am*m*x                    # Aca2 binding RNA
    propensities[6]  = dm*mx                     # Aca2 dissociating from RNA
    propensities[7]  = phi*m                     # Free RNA decay
    propensities[8]  = phi*mx                    # Bound RNA decay and release of Aca2
    propensities[9]  = beta*m                    # Aca2 translation from free RNA
    propensities[10] = beta*mx                   # Aca2 translation from bound RNA
    propensities[11] = delta*x                   # Aca2 decay
    propensities[12] = theta*m                   # Acr translation from free RNA
    propensities[13] = delta*y                   # Acr decay

# parameters to deactivate for different regulation scenarios

#                alpha, beta, theta, phi, delta, am, dm, au, du, utk, r, latency)
sampleConfig = [(1,     1,    1,     1,   1,     0,  1,  0,  1,  1,   1, 1), # no regulation
                (1,     1,    1,     1,   1,     1,  1,  0,  1,  1,   1, 1), # RNA-only
                (1,     1,    1,     1,   1,     0,  1,  1,  1,  1,   1, 1), # DNA-only
                (1,     1,    1,     1,   1,     1,  1,  1,  1,  1,   1, 1)] # full reg

#print(sampleConfig)


### Modeling lytic cycle
We implement the Gillespie Stochastic Simulation Algorithm in python of the model above. The system start with a single copy of DNA, and replicates up to 100 copies (we can change this parameter).


## Comparing modeling under three different regulation and without regulation

In [3]:
#@title Parameter setting for model

timescaleFactor = 60  # convert seconds to min

### parameters in human-friendly formats (used for plot axes labels) ###

# first-order mesoscopic
phi0 = 10         # units = min         # mRNA halflife    - converted elsewhere to rate
delta0 = 30      # units = min         # protein halflife - converted elsewhere to rate # not currently used

# second-order mesoscopic (1/(molecules x seconds))
alpha0 = 2       # units = 1/min/DNA   # transcription rate (50) unkown maximum production rate from Hill function
beta0 = 2        # units = 1/min/mRNA  # unkown translation rate for Aca2
theta0 = 20      # units = 1/min/mRNA  # translation rate of Acr. It does not matter because it only works as an amplification gain of the output

# second-order macroscopic (1/(molar x seconds))
# > assume 1 molecule per cell = 1 nM > second-order mesoscopic > pseudo first-order

# experimental estimates
kdu0 = 1.4        # units = nM          # Aca2-DNA dissociation constant (Kd)
kdm0 = 30.2       # units = nM          # Aca2-RNA dissociation constant (Kd)

# unknown approximations for K.on
au0 = 0.01       # units = 1/(nM.s) >> 1/(molecule.s) (1 nM = 1 molecule/cell)  # DNA association rate (K.on) # equivalent to 1x10^7/M/s
am0 = 0.01       # units = 1/(nM.s) >> 1/(molecule.s) (1 nM = 1 molecule/cell)  # RNA association rate (K.on)

# others...
moi = 1          # units = molecules   # Number of DNA molecules (unbound) at t=0
latency = 10      # units = min         # latent period before DNA replication begins
umax = 50        # units = molecules   # Total DNA (maximum)
r = 0.1          # units = ?           # DNA replication #QRY: what is this equivalent to? The doubling time for the DNA?


### parameter sweeping ###

paramsize = 5 # maximum number of elements for paramater sweeps

umax_v = (1, 10, 50, 100, 200)          # Varying total number of DNA

phi_v0 = (1, 3, 5, 10, 15)              # Varying mRNA decay rate (min)
beta_v0 = (1, 2.5, 5, 10, 25)           # Varying translation rate (Aca2) # QRY: need to dynamically adjust theta based on beta - theta should always be 10X beta?

moi_v = (1, 2, 5, 10, 20)               # Varying initial number of DNA molecules at t=0
r_v = (0.01, 0.05, 0.1, 0.2, 0.5)        # Varying DNA replication
alpha_v0 = (alpha0/10, alpha0/3.16 ,alpha0,alpha0*3.16,alpha0*10)     # Varying transcription rate
theta_v0 = (10, 20, 50, 100, 200)       # Varying translation rate (Acr)

kdu_v0 = (0.14, 0.5, 1.4, 5, 10)        # Varying dissociation constant (Aca2 to DNA)?
kdm_v0 = (3.2, 10, 30.2, 100, 302)      # Varying dissociation constant (Aca2 to RNA)?

au_v0 = (au0/10, au0/3.16, au0,au0*3.16,au0*10)      # Varying association rate (Aca2 to DNA) K.on
am_v0 = (au0/10, au0/3.16, au0, au0*3.16, au0*10)      # Varying association rate (Aca2 to RNA) K.on


### convert parameters to required format and scale ###

# convert RNA decay rate from t 1/2 to constant
phi = np.log(2)/phi0
phi_v = np.log(2)/phi_v0
delta = 0 # np.log(2)/delta0     # protein decay deactivated
#delta_v = np.log(2)/delta_v0

#adjust any other required parameters to the correct timescale or units
am = am0 * timescaleFactor # 1/sec to 1/min
au = au0 * timescaleFactor # 1/sec to 1/min

am_v = (tuple([val*timescaleFactor for val in am_v0]))  # 1/sec to 1/min
au_v = (tuple([val*timescaleFactor for val in au_v0]))  # 1/sec to 1/min


# convert Kd to K.off using mesoscopic K.on in 1/(molecules.min)
du = kdu0*au           # units = 1/min       # Aca2-DNA dissociation rate (K.off)
dm = kdm0*am           # units = 1/min       # Aca2-RNA dissociation rate (K.off)

du_v = (tuple([val*au for val in kdu_v0]))
dm_v = (tuple([val*am for val in kdm_v0]))


# unchanged units
alpha = alpha0
beta = beta0
theta = theta0

alpha_v = alpha_v0
beta_v = beta_v0
theta_v = theta_v0


### runtime settings

N = 2         # Sample size 200
n = 2           # number of cores on the computer (usuallly 2)
idAca = 4       # index of Aca counts in output array
idAcr = 5       # index of Acr counts in output array

time_points1 = np.linspace(0, 60, 121) # was 51 # QRY: what is best here given the fast dynamics of some processes (e.g. RNA decay)?
# initial state
population_0 = np.zeros(6, dtype=int)
population_0[0] = moi

f = 1 # times std for shaded region

#### don't run unregulated samples.... only implemented in parameter sweeps
numSamples=4
samplesToRun=(0,1,2,3)


###


# Aesthetics
palette1 = ('#fdd49e','#fc8d59','#b30000')
palette3 = ('black','blue')

paletteSweepAcr = ('#fdd49e','#fc8d59','#fc8d59','#f26066','#b30000') # lightest -> darkest
paletteSweepAca = ('#77ccff','#3388ff','#3388ff','#3388ff','#0044ff')
paletteSweepDNA = ('#94c58c','#429b46','#429b46','#429b46','#0a6921')
paletteSweepRNA = ('#e1affd','#ca8dfd','#ca8dfd','#ca8dfd','#b042ff')

paletteEndpointComp = ('black','#377eb8','#4daf4a','#e41a1c')

linestyle = ["solid","dashed"]

# Set the number of plot types (columns) for the parameter sweeps
plotCount = 6




label = ['AcrF8','Free Aca2']
title = ['unregulated','RNA regulation only','DNA regulation only','DNA/RNA regulation','+/- RNAreg. Dash = +RNAreg']

# Set up plots
fig_size = [600, 200] # for paper size (144, 110) # for larger size (144,144)
fig_size = [300, 200] # for paper size (144, 110) # for larger size (144,144)
x_range = (0, time_points1[-1])


In [4]:
def Get_data(vec_out, vec_in, option, fac):

    if option == 'mean':
        vec_out = np.vstack([vec_out, np.mean(y, axis=0)]) # Stack only the mean of y
    elif option == 'percentile':
        vec_out = np.vstack([vec_out, np.percentile(y, fac, axis=0),
                            np.mean(y, axis=0),
                            np.percentile(y, 100-fac, axis=0)]) # Stack 25%, 50% and 75% of y
    elif option == 'std': # standar deviation
        m = np.mean(y, axis = 0)
        sig = np.std(y, axis = 0)
        vec_out = np.vstack([vec_out, m - fac*sig,
                            m,
                            m + fac*sig]) # Stack 25%, 50% and 75% of y

    return vec_out



In [5]:
#@title run standard model/parameters

# initial state
population_0 = np.zeros(6, dtype=int)
population_0[0] = 1

args1 = (alpha, beta, theta, phi, delta, 0, dm, 0, du, umax, r, latency)
args2 = (alpha, beta, theta, phi, delta, am, dm, 0, du, umax, r, latency)
args3 = (alpha, beta, theta, phi, delta, 0, dm, au, du, umax, r, latency)
args4 = (alpha, beta, theta, phi, delta, am, dm, au, du, umax, r, latency)

# Running stochastic simulations

samples1 = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points1, size=N, args=args1, n_threads=n,progress_bar=False)
samples2 = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points1, size=N, args=args2, n_threads=n,progress_bar=False)
samples3 = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points1, size=N, args=args3, n_threads=n,progress_bar=True)
samples4 = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points1, size=N, args=args4, n_threads=n,progress_bar=True)

100%|██████████| 2/2 [00:03<00:00,  1.99s/it]


### Standard plot: RNA levels:

In [6]:
#@title Standard plot: RNA levels:


plots = []
for i in range(4):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          #x_range=x_range, y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"
    plots[i].axis.axis_label_text_font_size = "10pt"
    plots[i].axis.axis_label_text_font_style = "normal"

data = [samples1, samples2, samples3, samples4]
data_io = np.copy(time_points1) # define the variable to store the values for the CVS file

for k, samples in enumerate(data):
    y = samples[:,:,2] + samples[:,:,3] # samples[:,:,10][0]
    plots[k+0].step(time_points1, np.mean(y, axis=0), line_width=2, alpha= 1, line_join="bevel",color="black")
    data_io = Get_data(data_io, y, option="std", fac=1) # Extract data (only mean or percentile)

    y = samples[:,:,3] # samples[:,:,10][0]
    plots[k+0].step(time_points1, np.mean(y, axis=0), line_width=2, alpha= 1, line_join="bevel",color="red")
    data_io = Get_data(data_io, y, option="std", fac=1) # Extract data (only mean or percentile)

    y = samples[:,:,2] # samples[:,:,10][0]
    plots[k+0].step(time_points1, np.mean(y, axis=0), line_width=2, alpha= 1, line_join="bevel",color="green")
    data_io = Get_data(data_io, y, option="std", fac=1) # Extract data (only mean or percentile)

for i in range(4):
    plots[i].xaxis.axis_label = 'Time (min)'
    plots[i].yaxis.axis_label = 'Total RNA (# molecules)'
    plots[i].title = title[i]

plots[2].y_range=Range1d(0,10)
plots[3].y_range=Range1d(0,10)

output_notebook()
bokeh.io.show(bokeh.layouts.gridplot(plots,ncols=4))

DF = pd.DataFrame(data_io.transpose())
# save the dataframe as a csv file
DF.to_csv("data_RNA_levels.csv")

### Standard plot: Acr levels (mean) and rate of production:

In [7]:
#@title Standard plot: Acr levels (mean) and rate of production:
print('Grey regions correspond to +/- one standard deviation')
print('Use the toolbar to scroll/zoom plots that are off-scale')

y_range = (0, 1500)

plots = []
for i in range(10):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range , y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"


data = [samples1, samples2, samples3, samples4]
data_io_1 = np.copy(time_points1) # define the variable to store the values for the CVS file
data_io_2 = np.copy(time_points1) # define the variable to store the values for the CVS file

for k, samples in enumerate(data):
    y = samples[:,:,idAcr] # samples[:,:,10][0]
    m = np.mean(y, axis = 0)
    sig = np.std(y, axis = 0)
    plots[k+0].varea(time_points1, m - f*sig, m + f*sig ,color="black", alpha = 0.1)
    #plots[k].step(time_points1, np.mean(y, axis=0), line_width=2, alpha= 1, line_join="bevel",color="black")
    plots[k+0].line(time_points1, np.mean(y, axis=0), line_width=2, alpha= 1, line_join="bevel",color="black")

    data_io_1 = Get_data(data_io_1, y, option="std", fac=1) # Extract data (only mean or percentile)

    y1 = (y[:,1::] - y[:,0:-1:]) / (time_points1[2]-time_points1[1])
    plots[k+5+0].line(time_points1[0:-1:], np.mean(y1, axis=0), line_width=2, alpha= 1, line_join="bevel",color="red")

    data_io_2 = Get_data(data_io_2, y1, option="std", fac=1) # Extract data (only mean or percentile)

data = [samples3, samples4]
#data_io_3 = np.copy(time_points1) # define the variable to store the values for the CVS file
for k, samples in enumerate(data):
    y = samples[:,:,idAcr] # samples[:,:,10][0]
    m = np.mean(y, axis = 0)
    sig = np.std(y, axis = 0)
    #plots[8].varea(time_points1, m - f*sig, m + f*sig ,color="black", alpha = 0.1)
    #plots[k].step(time_points1, np.mean(y, axis=0), line_width=2, alpha= 1, line_join="bevel",color="black")
    plots[4].line(time_points1, np.mean(y, axis=0), line_width=2, alpha= 1, line_join="bevel",color=palette3[k])

    data_io_3 = Get_data(data_io_3, y, option="std", fac=1) # Extract data (only mean or percentile)

    y1 = (y[:,1::] - y[:,0:-1:]) / (time_points1[2]-time_points1[1])
    plots[9].line(time_points1[0:-1:], np.mean(y1, axis=0), line_width=2, alpha= 1, line_join="bevel",color=palette3[k])

    #data_io_3 = Get_data(data_io_3, y1, option="std", fac=1) # Extract data (only mean or percentile)

for i in range(5):
    plots[i].xaxis.axis_label = 'Time (min)'
    plots[i].yaxis.axis_label = label[0] + ' (# molecules)'
    plots[i].title = title[i]

    plots[i+5].xaxis.axis_label = 'Time (min)'
    plots[i+5].yaxis.axis_label = label[0] + ' (# molecules/min)'
    plots[i+5].title = title[i]
    plots[i+5].y_range = Range1d(0, 100)

output_notebook()
bokeh.io.show(bokeh.layouts.gridplot(plots,ncols=5))

DF = pd.DataFrame(data_io_1.transpose())
DF.to_csv("data_Acr_Levels.csv") # save the dataframe as a csv file

DF = pd.DataFrame(data_io_2.transpose())
DF.to_csv("data_Acr_rates.csv") # save the dataframe as a csv file

#DF = pd.DataFrame(data_io_3.transpose())
#DF.to_csv("data_Fig4_3.csv") # save the dataframe as a csv file

Grey regions correspond to +/- one standard deviation
Use the toolbar to scroll/zoom plots that are off-scale


NameError: ignored

# Sensitivity Analysis - Maximum total DNA

In [ ]:
# Vary DNA copy number (maximum)

plots = []
for i in range(8):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          #x_range=x_range, y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

data_io_1 = np.copy(time_points1) # define the variable to store the values for the CVS file
data_io_2 = np.copy(time_points1) # define the variable to store the values for the CVS file

for i, umaxk in enumerate(param_v):
    args1 = (alpha, beta, theta, phi, delta, 0, dm, 0, du, umaxk, r, latency)
    samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points1, size=N, args=args1, n_threads=n,progress_bar=False)

    y = samples[:,:,idAcr] # samples[:,:,10][0]
    m = np.mean(y, axis = 0)
    sig = np.std(y, axis = 0)

    plots[0].varea(time_points1, m - f*sig, m + f*sig ,color=paletteSweepAcr[i], alpha = 0.1)
    plots[0].line(time_points1, m, line_width=2, alpha= 1, line_join="bevel",color=paletteSweepAcr[i])

    data_io_1 = Get_data(data_io_1, y, option="std", fac=1) # Extract data (only mean or percentile)

    y1 = (y[:,1::] - y[:,0:-1:]) / (time_points1[2]-time_points1[1])
    m1 = np.mean(y1, axis = 0)
    sig1 = np.std(y1, axis = 0)
    plots[4].varea(time_points1[0:-1:], m1 - f*sig1, m1 + f*sig1 ,color=paletteSweepAcr[i], alpha = 0.1)
    plots[4].line(time_points1[0:-1:], m1, line_width=2, alpha= 1, line_join="bevel",color=paletteSweepAcr[i])

    data_io_2 = Get_data(data_io_2, y1, option="std", fac=1) # Extract data (only mean or percentile)

for i, umaxk in enumerate(param_v):
    args2 = (alpha, beta, theta, phi, delta, am, dm, 0, du, umaxk, r, latency)
    samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points1, size=N, args=args2, n_threads=n,progress_bar=False)

    y = samples[:,:,idAcr] # samples[:,:,10][0]
    m = np.mean(y, axis = 0)
    sig = np.std(y, axis = 0)

    plots[1].varea(time_points1, m - f*sig, m + f*sig ,color=paletteSweepAcr[i], alpha = 0.1)
    plots[1].line(time_points1, m, line_width=2, alpha= 1, line_join="bevel",color=paletteSweepAcr[i])

    data_io_1 = Get_data(data_io_1, y, option="std", fac=1) # Extract data (only mean or percentile)

    y1 = (y[:,1::] - y[:,0:-1:]) / (time_points1[2]-time_points1[1])
    m1 = np.mean(y1, axis = 0)
    sig1 = np.std(y1, axis = 0)
    plots[5].varea(time_points1[0:-1:], m1 - f*sig1, m1 + f*sig1 ,color=paletteSweepAcr[i], alpha = 0.1)
    plots[5].line(time_points1[0:-1:], m1, line_width=2, alpha= 1, line_join="bevel",color=paletteSweepAcr[i])

    data_io_2 = Get_data(data_io_2, y1, option="std", fac=1) # Extract data (only mean or percentile)


for i, umaxk in enumerate(param_v):
    args3 = (alpha, beta, theta, phi, delta, 0, dm, au, du, umaxk, r, latency)
    samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points1, size=N, args=args3, n_threads=n,progress_bar=False)

    y = samples[:,:,idAcr] # samples[:,:,10][0]
    m = np.mean(y, axis = 0)
    sig = np.std(y, axis = 0)

    plots[2].varea(time_points1, m - f*sig, m + f*sig ,color=paletteSweepAcr[i], alpha = 0.1)
    plots[2].line(time_points1, m, line_width=2, alpha= 1, line_join="bevel",color=paletteSweepAcr[i])

    data_io_1 = Get_data(data_io_1, y, option="std", fac=1) # Extract data (only mean or percentile)

    y1 = (y[:,1::] - y[:,0:-1:]) / (time_points1[2]-time_points1[1])
    m1 = np.mean(y1, axis = 0)
    sig1 = np.std(y1, axis = 0)
    plots[6].varea(time_points1[0:-1:], m1 - f*sig1, m1 + f*sig1 ,color=paletteSweepAcr[i], alpha = 0.1)
    plots[6].line(time_points1[0:-1:], m1, line_width=2, alpha= 1, line_join="bevel",color=paletteSweepAcr[i])

    data_io_2 = Get_data(data_io_2, y1, option="std", fac=1) # Extract data (only mean or percentile)


for i, umaxk in enumerate(param_v):
    args4 = (alpha, beta, theta, phi, delta, am, dm, au, du, umaxk, r, latency)
    samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points1, size=N, args=args4, n_threads=n,progress_bar=False)

    y = samples[:,:,idAcr] # samples[:,:,10][0]
    m = np.mean(y, axis = 0)
    sig = np.std(y, axis = 0)

    plots[3].varea(time_points1, m - f*sig, m + f*sig ,color=paletteSweepAcr[i], alpha = 0.1)
    plots[3].line(time_points1, m, line_width=2, alpha= 1, line_join="bevel",color=paletteSweepAcr[i])

    data_io_1 = Get_data(data_io_1, y, option="std", fac=1) # Extract data (only mean or percentile)

    y1 = (y[:,1::] - y[:,0:-1:]) / (time_points1[2]-time_points1[1])
    m1 = np.mean(y1, axis = 0)
    sig1 = np.std(y1, axis = 0)
    plots[7].varea(time_points1[0:-1:], m1 - f*sig1, m1 + f*sig1 ,color=paletteSweepAcr[i], alpha = 0.1)
    plots[7].line(time_points1[0:-1:], m1, line_width=2, alpha= 1, line_join="bevel",color=paletteSweepAcr[i])

    data_io_2 = Get_data(data_io_2, y1, option="std", fac=1) # Extract data (only mean or percentile)


for i in range(4):
    plots[i].xaxis.axis_label = 'Time (min)'
    plots[i].yaxis.axis_label = label[0] + ' (# molecules)'
    plots[i].title = title[i]

    plots[i+4].xaxis.axis_label = 'Time (min)'
    plots[i+4].yaxis.axis_label = label[0] + ' (# molecules/min)'
    plots[i+4].title = title[i]
    plots[i+4].y_range = Range1d(0, 100)

output_notebook()
bokeh.io.show(bokeh.layouts.gridplot(plots,ncols=4))

DF = pd.DataFrame(data_io_1.transpose())
DF.to_csv("data_DNA-Capacity_Acr_Levels_.csv") # save the dataframe as a csv file

DF = pd.DataFrame(data_io_2.transpose())
DF.to_csv("data_DNA-Capacity_Acr_rates.csv") # save the dataframe as a csv file

Loading BokehJS ...